# Demo: Delta Live Tables (DLT) com Yellow Taxi

Este notebook demonstra como utilizar o **Delta Live Tables (DLT)** no Databricks para criar pipelines de ingestão e transformação de dados confiáveis, auditáveis e fáceis de manter. O DLT automatiza tarefas como gerenciamento de qualidade, versionamento e monitoramento dos dados.

Vamos usar o dataset do yellow taxi, já utilizado no teste do Auto Loader, para mostrar como criar tabelas bronze (raw), silver (limpas) e gold (agregadas) com DLT.

## O que é Delta Live Tables (DLT)?

O Delta Live Tables é uma framework de pipelines declarativos da Databricks que permite:

- Declarar tabelas e transformações como código Python ou SQL.
- Gerenciar automaticamente a qualidade dos dados (expectations).
- Versionar e monitorar pipelines de dados.
- Automatizar ingestão, transformação e atualização de tabelas Delta.

DLT é ideal para pipelines de ingestão incremental, ETL e preparação de dados para analytics e machine learning.

## Pipeline DLT: Yellow Taxi

Vamos criar um pipeline DLT com três camadas:

- **Bronze:** ingestão bruta dos dados Parquet do yellow taxi.
- **Silver:** limpeza e padronização dos dados.
- **Gold:** agregação de métricas para análise.

In [ ]:
import dlt
from pyspark.sql.functions import col, trunc, year, month

# Caminho dos dados CDC do yellow taxi (Parquet)
cdc_path = "abfss://taxi@hfotaxinyc.dfs.core.windows.net/cdc/yellow"

In [ ]:
@dlt.table(
  comment="Tabela bronze: ingestão bruta dos arquivos Parquet do yellow taxi."
)
def bronze_yellow_taxi():
    return (
        spark.read.format("parquet").load(cdc_path)
    )

In [ ]:
@dlt.table(
  comment="Tabela silver: dados limpos e padronizados do yellow taxi."
)
@dlt.expect("pickup_date_not_null", "tpep_pickup_datetime IS NOT NULL")
def silver_yellow_taxi():
    df = dlt.read("bronze_yellow_taxi")
    return (
        df
        .withColumn("pickup_year_month", trunc("tpep_pickup_datetime", "MM"))
        .withColumn("pickup_year", year("tpep_pickup_datetime"))
        .withColumn("pickup_month", month("tpep_pickup_datetime"))
        .filter(col("passenger_count") > 0)
    )

In [ ]:
@dlt.table(
  comment="Tabela gold: agregação de corridas e receita por mês."
)
def gold_yellow_taxi_monthly():
    df = dlt.read("silver_yellow_taxi")
    return (
        df.groupBy("pickup_year", "pickup_month")
          .agg(
              {"*": "count", "total_amount": "sum"}
          )
          .withColumnRenamed("count(1)", "num_trips")
          .withColumnRenamed("sum(total_amount)", "total_revenue")
    )

## Como executar este notebook

1. Crie um pipeline Delta Live Tables no Databricks e aponte para este notebook.
2. Execute o pipeline para ver as tabelas bronze, silver e gold sendo criadas e atualizadas automaticamente.
3. Use as tabelas gold para análises e dashboards.

**DLT** garante qualidade, rastreabilidade e automação no seu pipeline de dados!